In [18]:
!git clone --branch master_colab https://github.com/aravindadithya/DLR

^C


In [1]:
!pip install visdom
#python -m visdom.server -port 8097

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 40.6 MB/s eta 0:00:0000:0100:01
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=c195827cd2740cdc366e0d5856b5bf539df86858edd94b1184d8d37f1dabccf3
  Stored in directory: /root/.cache/pip/wheels/37/6c/38/64eeaa310e325aacda723e6df1f79ab5e9f31ba195264e04a8
Successfully built visdom


In [3]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 735.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.1 MB/s eta 0:00:0000:0100:01


In [3]:
!python -m visdom.server -port 8097

/bin/bash: line 1: python: command not found


In [1]:
import sys
import os
current_dir = os.getcwd()
print(current_dir)

#parent_dir = os.path.join(current_dir, 'DLR')
parent_dir = os.path.dirname(os.path.dirname(current_dir))
print(parent_dir)
model_dir = os.path.join(parent_dir, 'trained_models', 'CIFAR', 'model2', 'nn_models/')
'''
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model4\\nn_models\\'
'''
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

/workspace/DLR/experiments/rfm_cifar
/workspace/DLR


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_gcnn as agc
from utils import trainer as tr
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
#import rfm
import numpy as np
from trained_models.CIFAR.model2 import trainer as t
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M
from groupy.gconv.make_gconv_indices import *
from copy import deepcopy
from torch.nn.functional import pad
from torch.func import jacrev

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


CURR /workspace/DLR/experiments/rfm_cifar
/workspace/DLR/utils
CURR /workspace/DLR/experiments/rfm_cifar
/workspace/DLR/utils
/workspace/DLR/experiments/rfm_cifar
/workspace/DLR
/workspace/DLR/trained_models/CIFAR/model2/nn_models/
/workspace/DLR


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")


Using device: cuda:0


In [4]:
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

ModuleNotFoundError: No module named 'torch_xla'

In [4]:
torch.cuda.empty_cache()

In [5]:
net= t.get_untrained_net()
print(net)
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Total number of trainable parameters in ResNet: {total_params}")

ResNet(
  (features): Sequential(
    (0): P4MConvZ2()
    (1): BatchNorm3d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1): BatchNorm3d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): P4MConvP4M()
        (4): BatchNorm3d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (lrelu): ReLU()
    )
    (4): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1): BatchNorm3d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): P4MConvP4M()
        (4): BatchNorm3d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (lrelu): ReLU()
    )
    (5): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1)

In [6]:
trainloader, valloader, testloader = t.get_loaders()
#net= t.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir+'cifar_gcnn_trained_nn.pth'):
    checkpoint = torch.load(model_dir+'cifar_gcnn_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
t.train_net(device)

Model weights loaded successfully.
Model weights loaded successfully.
NUMBER OF PARAMS:  31418369
Time:  80.25323534011841
Epoch:  0 Train Loss:  0.009932412 Test Loss:  0.41551998 Train Acc:  99.97 Test Acc:  90.14 Best Val Acc:  92.69 Best Val Loss:  0.27429533 Best Test Acc:  90.14 Best Test Loss:  0.41551998
Time:  80.21320676803589
Epoch:  1 Train Loss:  0.008190602 Test Loss:  0.42014205 Train Acc:  99.975 Test Acc:  90.02 Best Val Acc:  92.69 Best Val Loss:  0.27429533 Best Test Acc:  90.14 Best Test Loss:  0.41551998
Time:  80.20716142654419
Epoch:  2 Train Loss:  0.0071237274 Test Loss:  0.42574906 Train Acc:  99.985 Test Acc:  90.04 Best Val Acc:  92.69 Best Val Loss:  0.27429533 Best Test Acc:  90.14 Best Test Loss:  0.41551998


In [ ]:
tr.visualize_predictions(net, testloader, range(10), device, num_images=36)